# Data Workshop: Custom Experiment Analysis with Optimizely Data Services

## The experiment

Show experiment variations

## The challenge

What impact did this have on call center volumes?

## What we're going to do

1. Download Optimizely decision data for our experiment
2. Join that data with our call center log data
3. Compute several useful intermediate experiment datasets
4. Compute a set of experiment metrics
5. Compute sequential p-values and confidence intervals using Optimizely Stats Engine as a service
6. Render a simple experiment results report

## Downloading Optimizely Enriched Event data

In [15]:
%env OPTIMIZELY_DATA_DIR=./opticon_data
%env OPTIMIZELY_API_TOKEN=2:bqZXaNE24MFUlhyGFrKKY9DMA-G02xoou7fR0nQlQ3bT89uvjtF8

env: OPTIMIZELY_DATA_DIR=./opticon_data
env: OPTIMIZELY_API_TOKEN=2:bqZXaNE24MFUlhyGFrKKY9DMA-G02xoou7fR0nQlQ3bT89uvjtF8


### Download experiment decision data

In [18]:
!oevents load --type decisions --experiment 18156943409 --date 2020-05-25

download: s3://optimizely-events-data/v1/account_id=596780373/type=decisions/date=2020-05-25/experiment=18156943409/part-00000-f1c6c551-2762-401d-bc5e-c4dfb9cfd078.c000.snappy.parquet to opticon_data/type=decisions/date=2020-05-25/experiment=18156943409/part-00000-f1c6c551-2762-401d-bc5e-c4dfb9cfd078.c000.snappy.parquet


### Download conversion event data

In [19]:
!oevents load --type events --event add_to_cart --date 2020-05-25

download: s3://optimizely-events-data/v1/account_id=596780373/type=events/date=2020-05-25/event=add_to_cart/part-00000-a60ce976-cfb4-4459-b49d-ae45ef299a2b.c000.snappy.parquet to opticon_data/type=events/date=2020-05-25/event=add_to_cart/part-00000-a60ce976-cfb4-4459-b49d-ae45ef299a2b.c000.snappy.parquet


# Enrich and compute intermediate experiment datasets

We'll use logic encoded in three other notebooks in this directory to process our downloaded data.

In [20]:
from lib.util import get_or_create_spark_session

spark = get_or_create_spark_session()

In [21]:
%run ./00_enriching_decision_data.ipynb

Successfully authenticated to Optimizely.
Found these experiment IDs in the loaded decision data:
    18156943409


/opt/anaconda3/envs/optimizelylabs/lib/python3.7/site-packages/bravado_core/spec.py:464: Warning: uuid format is not registered with bravado-core!
  category=Warning,
/opt/anaconda3/envs/optimizelylabs/lib/python3.7/site-packages/bravado_core/spec.py:464: Warning: url format is not registered with bravado-core!
  category=Warning,


[  experiment_id variation_id    experiment_name variation_name  \
0   18156943409  18174970251  free_shipping_exp        control   
1   18156943409  18112613000  free_shipping_exp  free_shipping   

  reference_variation_id  
0            18174970251  
1            18174970251  ]


### Experiment Units

**Experiment units** are the individual units that are exposed to a control or treatment in the course of an online experiment.  In most online experiments, subjects are website visitors or app users. However, depending on your experiment design, treatments may also be applied to individual user sessions, service requests, search queries, etc. 

In [7]:
%run ./01_computing_experiment_units.ipynb

Let's take a look at our intermediate experiment units dataset:

In [8]:
spark.sql("""
    SELECT 
        experiment_name, 
        variation_name, 
        count(*) as unit_count
    FROM 
        experiment_units 
    GROUP BY
        experiment_name,
        variation_name
""")

experiment_name,variation_name,unit_count
free_shipping_exp,free_shipping,4487
free_shipping_exp,control,4514


### Experiment Events

An **experiment event** is an event, such as a button click or a purchase, that was influenced by an experiment.  We compute this view by isolating the conversion events triggered during a finite window of time (called the _attribution window_) after a visitor has been exposed to an experiment treatment.

In [9]:
%run ./02_computing_experiment_events.ipynb

In [10]:
spark.sql("""
    SELECT 
        experiment_name,
        variation_name,
        event_name, 
        count(1) 
    FROM 
        experiment_events 
    GROUP BY 
        experiment_name, 
        variation_name,
        event_name
""")

experiment_name,variation_name,event_name,count(1)
free_shipping_exp,free_shipping,add_to_cart,2577
free_shipping_exp,control,add_to_cart,2655


In [27]:
spark.sql("SELECT * FROM experiment_events LIMIT 2")

experiment_id,experiment_name,variation_id,variation_name,uuid,timestamp,process_timestamp,visitor_id,session_id,account_id,experiments,entity_id,attributes,user_ip,user_agent,referer,event_type,event_name,revenue,value,quantity,tags,revision,client_engine,client_version,date,event
18156943409,free_shipping_exp,18174970251,control,22A0270C-EF16-4369-A674-89650D02CA1E,2020-05-25 15:29:24.445,2020-05-25 15:29:16.741,visitor_1590445764445,1144071362,596780373,"[[18149940006, 18156943409, 18174970251, false]]",15776040040,"[[100,, browserId, ie], [300,, device, desktop], [600,, source_type, campaign], [200,, campaign, winter campaign], [,...",75.111.77.0,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/53...",https://app.optimizely.com/,null,add_to_cart,0,0.0,null,[],null,ricky/fakedata.pwned,1.0.0,2020-05-25,add_to_cart
18156943409,free_shipping_exp,18112613000,free_shipping,FC0471CB-FD10-4251-AA4D-6725F9A5AE42,2020-05-25 15:32:47.245,2020-05-25 15:29:20.394,visitor_1590445967245,576740992,596780373,"[[18149940006, 18156943409, 18112613000, false]]",15776040040,"[[100,, browserId, ff], [300,, device, tablet], [600,, source_type, search], [200,, campaign, frequent visitors], [, ...",75.111.77.0,"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_4) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/81.0.4044.138 Safari/53...",https://app.optimizely.com/,null,add_to_cart,0,0.0,null,[],null,ricky/fakedata.pwned,1.0.0,2020-05-25,add_to_cart


## Compute experiment observations

Next we'll compute a versatile dataset for experiment analysis: _experiment observations_.

In this dataset we map each experiment subject (visitor) to a set of numerical observations made in the course of the experiment.  These observations may then be aggregated to compute _metrics_ we can analyze to measure the impact of our experiment.

We'll start by creating an "empty" `observations` dataset from our subjects dataset.

Now we'll define a handy function for adding a new set of observations to our view. This function takes a `name` and a `query` as input and executes the following transformations:
1. Execute the supplied `query` on the `experiment_events` view
2. Joins the resulting observations with our `observations` view and updates it

This function builds our `observations` dataset, one column at time.

In [20]:
spark.sql(f"""
    CREATE OR REPLACE TEMPORARY VIEW observations AS
    SELECT * FROM experiment_units
""")

def add_observations(metric_name, query):
    query_frame = spark.sql(query)
    query_frame.createOrReplaceTempView("new_observations")
    spark.sql(f"""
        CREATE OR REPLACE TEMPORARY VIEW observations AS
            SELECT
                o.*,
                COALESCE(n.observation, 0) as `{metric_name}`
            FROM
                observations o LEFT JOIN new_observations n on o.{UNIT_ID_FIELD} = n.{UNIT_ID_FIELD}
    """)
    # Return a sample of the new observations dataframe
    return spark.sql(f"SELECT {UNIT_ID_FIELD}, variation_name, timestamp, `{metric_name}` FROM observations LIMIT 5")

Now we'll define a set of observations by executing simple queries on our experiment events.  Each query computes a single _observation_ for each subject.

### Metric: Add to cart unique Conversions

In this query we compute the number of unique conversions on a particular event. The resulting observation should be `1` if the visitor triggered the event in question during the _attribution window_ and `0` otherwise.  

Since _any_ visitor who triggered an appropriate experiment event should be counted, we can simply select a `1`. 

In [21]:
## Unique conversions on the "add to cart" event.
add_observations('Add to cart conversion rate',
    f"""
        SELECT
            {UNIT_ID_FIELD},
            1 as observation
        FROM
            experiment_events
        WHERE
            event_name = 'add_to_cart'
        GROUP BY
            {UNIT_ID_FIELD}
    """
)

visitor_id,variation_name,timestamp,Add to cart conversion rate
visitor_1590445669165,free_shipping,2020-05-25 15:27:49.165,0
visitor_1590445676365,control,2020-05-25 15:27:56.365,0
visitor_1590445778365,control,2020-05-25 15:29:38.365,1
visitor_1590445862125,free_shipping,2020-05-25 15:31:02.125,1
visitor_1590445923325,free_shipping,2020-05-25 15:32:03.325,0


### Metric: Add to cart Conversions per visitor

In this query we compute the number of unique conversions on a particular event. The resulting observation should be `1` if the visitor triggered the event in question during the _attribution window_ and `0` otherwise.  

Since _any_ visitor who triggered an appropriate experiment event should be counted, we can simply select a `1`. 

In [22]:
## Unique conversions on the "capture" event.
add_observations('Add to cart conversions per visitor',
    f"""
        SELECT
            {UNIT_ID_FIELD},
            count(1) as observation
        FROM
            experiment_events
        WHERE
            event_name = "add_to_cart"
        GROUP BY
            {UNIT_ID_FIELD}
    """
)

visitor_id,variation_name,timestamp,Add to cart conversions per visitor
visitor_1590445669165,free_shipping,2020-05-25 15:27:49.165,0
visitor_1590445676365,control,2020-05-25 15:27:56.365,0
visitor_1590445778365,control,2020-05-25 15:29:38.365,1
visitor_1590445862125,free_shipping,2020-05-25 15:31:02.125,1
visitor_1590445923325,free_shipping,2020-05-25 15:32:03.325,0


## Extract useful views from observation data

Extract metric names, experiment IDs, reference variation ids, and metric-specific observation data from our loaded observation data.

In [23]:
from lib.util import map_from_columns

observations = spark.sql("SELECT * FROM observations")
experiment_names = map_from_columns(observations, "experiment_id", "experiment_name")
reference_variation_ids = map_from_columns(observations, "experiment_id", "reference_variation_id")
metric_names = observations.drop(*experiment_units.columns).columns

def get_variation_names_from_observations(exp_id):
    exp_observations = spark.sql(f"SELECT * FROM observations WHERE experiment_id='{exp_id}'")
    return map_from_columns(exp_observations, "variation_id", "variation_name")

In [24]:
def get_aggregate_metric_observations_df(experiment_id, metric_name, interval_resolution="HOUR"):
    return spark.sql(f"""
        SELECT
            '{metric_name}' as metric_name,
            experiment_id,
            experiment_name,
            variation_id,
            variation_name,
            reference_variation_id,
            DATE_TRUNC('{interval_resolution}', timestamp) as interval_timestamp,
            count(1) as unit_count,
            SUM(`{metric_name}`) as unit_observation_sum,
            SUM(`{metric_name}`*`{metric_name}`) as unit_observation_sum_of_squares
        FROM
            observations
        WHERE
            experiment_id = '{experiment_id}'
        GROUP BY
            metric_name,
            experiment_id,
            experiment_name,
            variation_id,
            variation_name,
            reference_variation_id,
            interval_timestamp
        ORDER BY
            interval_timestamp ASC
    """).toPandas()

## Rendering a results report

In this section we'll define helper functions for rendering an experiment report.

In [25]:
from IPython.display import display, Markdown, HTML
from lib.report import render
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd
from lib import ses_client

pio.renderers.default = "svg"

def plot_metric_stats(ses_results_df, metric_name, field, variation_names):
    fig = go.Figure()

    for v in ses_results_df.variation_id.unique():
        fig.add_trace(
            go.Scatter(
                x=ses_results_df[ses_results_df.variation_id == v].interval_timestamp,
                y=ses_results_df[field],
                mode="lines",
                line=dict(shape="hv"),
                name=f"variation: {variation_names.get(v, v)}",
                showlegend=True,
            )
        )
    fig.update_layout(
        yaxis_title=field,
        title=f"{metric_name}: {field}",
        #yaxis=dict(range=[min(ses_results_df[field]), max(ses_results_df[field])]),
        xaxis_title="Timestamp",
    )
    
    return fig

def render_experiment_report(experiment_id, reference_variation_id, stats_fields_to_plot=None, interval_resolution="HOUR"):
    clear_output()
    
    stats_fields_to_plot = stats_fields_to_plot or []
    
    metric_observations_dfs = [
        get_aggregate_metric_observations_df(experiment_id, metric_name, interval_resolution)
        for metric_name in metric_names
    ]
    
    seq_stats_dfs = ses_client.make_ses_request(metric_observations_dfs, reference_variation_id)
    
    variation_names = get_variation_names_from_observations(experiment_id)
    
    display(Markdown(f"# Experiment Report: `{experiment_names.get(experiment_id, experiment_id)}`"))
    for i, metric_name in enumerate(metric_names):
        table_html = render.render_se_metric_overview_table(
            observations_timeseries=metric_observations_dfs[i],
            statistics=seq_stats_dfs[i],
            reference_variation_id=reference_variation_id,
            metric_name=metric_name,
            variation_names=variation_names
        )
        display(HTML(table_html))
        
        for field in stats_fields_to_plot:
            fig = plot_metric_stats(seq_stats_dfs[i], metric_name, field, variation_names)
            fig.show(width=1200, height=500, scale=1)

Next we'll use IPython's `interactive` module to render a dynamic experiment report

In [26]:
from ipywidgets import widgets, interact, interactive, Dropdown

experiment_id_dropdown = Dropdown(
    description='Experiment',
    options=[(exp_name, exp_id) for exp_id, exp_name in experiment_names.items()],
    style={'description_width': 'initial'},
)

hidden_experiment_id_select = Dropdown(
    options=[(exp_name, exp_id) for exp_id, exp_name in experiment_names.items()]
)
hidden_experiment_id_select.layout.display = 'none'

reference_variation_id_dropdown = Dropdown(
    description="Baseline variation",
    style={'description_width': 'initial'},
)

stats_plots_select = widgets.SelectMultiple(
    description='Plot vs time',
    options=[
        ("Corrected p-value", "corrected_p_value"), 
        ("Estimated Lift", "lift_estimate")
    ],
    value=[],
    disabled=False
)

interval_resolution_dropdown = Dropdown(
    description="Aggregation interval",
    options=[("Daily", "DAY"), ("Hourly", "HOUR"), ("Minutely", "MINUTE")],
    style={'description_width': 'initial'},
    value="HOUR"
)

def select_experiment(experiment_id):
    
    # clear_output()
    
    hidden_experiment_id_select.value = experiment_id
    variation_names = get_variation_names_from_observations(experiment_id)
    reference_variation_id_dropdown.options = [(var_name, var_id) for var_id, var_name in variation_names.items()]
    reference_variation_id_dropdown.value = reference_variation_ids[experiment_id]
    
    u = interactive(
        render_experiment_report, 
        experiment_id=hidden_experiment_id_select,
        reference_variation_id=reference_variation_id_dropdown,
        stats_fields_to_plot=stats_plots_select,
        interval_resolution=interval_resolution_dropdown
    )
    
    display(u)

w = interactive(
    select_experiment,
    experiment_id=experiment_id_dropdown
)

display(w)

interactive(children=(Dropdown(description='Experiment', options=(('free_shipping_exp', '18156943409'),), styl…

## How to run this notebook

This notebook lives in the [Optimizely Labs](http://github.com/optimizely/labs) repository.  You can download it and everything you need to run it by doing one of the following
- Downloading a zipped copy of this Lab directory on the [Optimizely Labs page](https://www.optimizely.com/labs/computing-experiment-subjects/)
- Downloading a [zipped copy of the Optimizely Labs repository](https://github.com/optimizely/labs/archive/master.zip) from Github
- Cloning the [Github respository](http://github.com/optimizely/labs)

Once you've downloaded this Lab directory (on its own, or as part of the [Optimizely Labs](http://github.com/optimizely/labs) repository), follow the instructions in the `README.md` file for this Lab.